# Imports

In [ ]:
# Allows you to make changes in the library files and include them in the 
# notebook without restarting the kernel
%load_ext autoreload
%autoreload 2

In [ ]:
# Python Library Imports
import numpy as np
import random
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as animation
import math
import queue
import time 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils
from torch.autograd import Variable
from sklearn.utils import shuffle
from scipy.integrate import ode, odeint
%matplotlib inline

### REPO IMPORTS ###
import astar_tree_search
import baseline_planners
import dataset_generation
import deadbeat_net
import hopper
import models
from models import StepSequenceModelConv
import rnn_planners
import terrain_utils
import test_utils
import training_utils
import utils
import evaluation

Defining Variables

In [ ]:
glob_dir = "./"
dataset_dir = glob_dir + "datasets/"
model_dir = glob_dir + "models/"
anim_dir = glob_dir + "animations/"
pics_dir = glob_dir + "images/"
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("using GPU")
else:
    device = torch.device("cpu")
    print("warning: using cpu")

using GPU


# Dataset Generation

### Multithreaded dataset generation with Angle-space planner


In [ ]:
def cost_function(x_flight, neighbors, goal, p):
  x_pos = x_flight[0]
  x_vel = x_flight[2]
  spread = 0
  # calculate spread of neighbors
  for n in range(len(neighbors)):
    spread += np.abs(neighbors[n][0] - x_pos)/len(neighbors)
   
  return 0.5 * np.abs(x_pos - goal[0]) + 0.5 * np.abs(x_vel - goal[1]) + 1 * spread

robot = hopper.Hopper(hopper.Constants())
max_seq_length = 15

def gen_args(seed):
  return (robot, 24, 8, 8, 8, 0.6, cost_function, True, 0.1, False, seed)

num_proc = 15
all_args = [gen_args(np.random.randint(1, 100)) for i in range(num_proc)]
p = mp.Pool(processes = num_proc)
return_val_array = p.starmap(dataset_generation.generateRandomSequences, all_args)
all_initial_states = []
all_sequences = []

for r in return_val_array:
  initial_states = r[0]
  sequences = r[1]
  for i in range(len(initial_states)):
    if max_seq_length > 0 and len(sequences[i]) < max_seq_length:
      all_initial_states.append(initial_states[i])
      all_sequences.append(sequences[i])

In [ ]:
print(len(all_sequences))
lens = [0 for i in range(50)]
for i in range(len(all_sequences)):
  lens[len(all_sequences[i])] += 1

In [ ]:
suffix = "360_120_spread3.npy"
suffix_oh = "360_120_spread3_oh.npy"

np.save(dataset_dir + "rnn_inits_" + suffix, all_initial_states)
np.save(dataset_dir + "rnn_seqs_" + suffix, all_sequences)
sequences_oh = utils.oneHotEncodeSequences(all_sequences)
np.save(dataset_dir + "rnn_seqs_" + suffix_oh, sequences_oh)

# Training Convolutional-Input Recurrent Net

Loading & Processing Data

In [ ]:
suffix = "360_120_spread3.npy"
suffix_oh = "360_120_spread3_oh.npy"

init_states = np.load(dataset_dir + "rnn_inits_" + suffix, allow_pickle = True)
sequences_oh = np.load(dataset_dir + "rnn_seqs_" + suffix_oh, allow_pickle = True)
sequences = np.load(dataset_dir + "rnn_seqs_" + suffix, allow_pickle = True)

In [ ]:
init_apexes, sequences_and_terrains = utils.concatenateTerrainsAndOHSteps(init_states, sequences_oh)
all_batches = utils.createDataBatches(sequences_and_terrains, init_apexes, batch_size = 64, train_pct = 0.9)
train_seq_batches = all_batches[0]
train_iv_batches = all_batches[1]
test_seq_batches = all_batches[2]
test_iv_batches = all_batches[3]

Training Convolutional-Input LSTM

In [ ]:
 # Fixed parameters based on the initial state/terrain sizes 
init_data_dim = 3
input_size = 110
output_size = 110

# Hyperparameters
hidden_dim = 110
n_layers = 2
use_lstm = True
ksize = 7

model = StepSequenceModelConv(init_dim = init_data_dim,
                              input_size = input_size,
                              output_size = output_size,
                              hidden_dim = hidden_dim,
                              n_layers = n_layers,
                              use_lstm = use_lstm,
                              ksize = ksize)

In [ ]:
n_epochs = 15
lr = 1e-3
model = training_utils.trainConvRNN(model,
                                    n_epochs,
                                    lr,
                                    train_seq_batches,
                                    train_iv_batches,
                                    device)

In [ ]:
 if use_lstm:
  model_name = "ConvLSTM_trained_"
else:
  model_name = "ConvRNN_trained_"
model_name += str(n_layers) + "x" + str(hidden_dim) + "_" + suffix[:-4]
torch.save(model, model_dir + model_name)

Test loss of model

In [ ]:
model = model.eval()
training_utils.convRNNValidation(model, test_seq_batches, test_iv_batches, device)

# Loading Pretrained Models and Controller

In [ ]:
db_net = deadbeat_net.DeadbeatNet()
db_net.load_state_dict(torch.load(glob_dir + "deadbeat_net_state_1104", map_location = device))
db_net = db_net.to(device)

step_controller = deadbeat_net.DeadbeatStepController(hopper.constants, db_net, ks = 0.0, device = device)
model = torch.load(model_dir + "ConvLSTM_trained_2x110_360_120_spreadhalf", map_location = device)

# Generate/Load Test Suites

Generate test matrix

In [ ]:
num_apexes = 5
ditch_profile = [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]
step_profile = [2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6]

ditch_test_matrix = test_utils.generateTestMatrix(ditch_profile, num_apexes)
step_test_matrix = test_utils.generateStepTestMatrix(step_profile, num_apexes)

import pickle
fname = "ditch_test_matrix_" + str(len(ditch_profile)) + str("x") + str(num_apexes)
with open(fname, "wb") as f:
  pickle.dump(ditch_test_matrix, f)

fname = "step_test_matrix_" + str(len(step_profile)) + str("x") + str(num_apexes) + "_4"
with open(fname, "wb") as f:
  pickle.dump(step_test_matrix, f)

Load Test Matrix

In [ ]:
import pickle

fname = "ditch_test_matrix_15x5"
with open(fname, 'rb') as f:
  ditch_test_matrix = pickle.load(f)

fname = "step_test_matrix_15x5_4"
with open(fname, 'rb') as f:
  step_test_matrix = pickle.load(f)

# Evaluate Planners on Test Suite

### Receding Horizon A* Planner

In [ ]:
robot = hopper.Hopper(hopper.Constants())
astar_planner = baseline_planners.AStarPlanner(robot, num_samples = 20, fallback_samples = 30,
                                               max_speed = 4, cost_matrix = [1, 1, 2])
tr = evaluation.testSearchPlannerOnMatrix(robot, astar_planner, step_controller,
                                          step_test_matrix, friction = 0.8, time_to_replan = 3, tstep = 0.01)

print()
tr.printMetrics()
readable_failures = {hopper.sim_codes_rev[i]:tr.failure_cases[i] for i in tr.failure_cases.keys()}
print(readable_failures)

### Footstep Space A* Planner

In [ ]:
# Could use different robot here to evaluate robustness to model perturbations
robot = hopper.Hopper(hopper.Constants())
foot_astar_planner = baseline_planners.FootSpaceAStarPlanner(robot, horizon = 5, spacing = 0.25,
                                                             cost_matrix = [1.0, 0.5, 0.5],
                                                             step_controller = step_controller)

In [ ]:
perturbed_controller = deadbeat_net.PerturbedStepController(step_controller, 0.00)
tr = evaluation.testSearchPlannerOnMatrix(robot, foot_astar_planner,
                                          step_controller, step_test_matrix,
                                          friction = 0.8, time_to_replan = 3,
                                          tstep = 0.01)
print()
tr.printMetrics()
readable_failures = {hopper.sim_codes_rev[i]:tr.failure_cases[i] for i in tr.failure_cases.keys()}
print(readable_failures)

Generate Animations of footstep planner

In [ ]:
terrain_idx = 5
apex_idx = 0

code, body_poses, foot_poses, plans, num_steps_hit, total_odes, _ = evaluation.testPlannerSingle(robot, foot_astar_planner, step_controller,
                                                                                              ditch_test_matrix, 2,
                                                                                              terrain_idx, apex_idx, friction = 0.8)

# Generate the animation from the matrix
funcs = ditch_test_matrix.getFunctions()
func = funcs[terrain_idx]
fig, ax = plt.subplots()
terrain_utils.plot_terrain(ax, 0, 8, func)
name = anim_dir + "footstep_planner_" + str(terrain_idx) + "_" + str(apex_idx) + ".mp4"
# utils.animateMovingXAxis(body_poses, foot_poses, anim_name = name, plans = plans, terrain_func = func, nogo = None, fps = 50)

### Receding-Horizon Heuristic Stride Planner

In [ ]:
h_planner = baseline_planners.HeuristicPlanner(stride = 1.0, buffer = 0.2)
robot = hopper.Hopper(hopper.Constants())
tr = evaluation.testSearchPlannerOnMatrix(robot, h_planner, step_controller, step_test_matrix,
                                          friction = 0.8, time_to_replan = 3, tstep = 0.01)

print()
tr.printMetrics()
readable_failures = {hopper.sim_codes_rev[i]:tr.failure_cases[i] for i in tr.failure_cases.keys()}
print(readable_failures)

### Receding-Horizon RNN Guided A*

In [ ]:
robot = hopper.Hopper(hopper.Constants())
rnn_planner = rnn_planners.ConvRNNPlanner(model, device, min_limit = -3, T = 3)
rnn_astar_planner = rnn_planners.RNNAStarPlanner(robot, rnn_planner, step_controller,
                                                 num_samples = 3, fallback_samples = 5,
                                                 cost_matrix = [1, 0.5, 0.0])

In [ ]:
# Wraps the ConvRNNPlanner in a tree search.
perturbed_controller = deadbeat_net.PerturbedStepController(step_controller, 0.00)
test_results2 = evaluation.testSearchPlannerOnMatrix(robot, rnn_astar_planner,
                                                     perturbed_controller, ditch_test_matrix,
                                                     friction = 0.8, time_to_replan = 3,
                                                     tstep = 0.01)

print()
test_results2.printMetrics()
readable_failures = {hopper.sim_codes_rev[i]:test_results2.failure_cases[i] for i in test_results2.failure_cases.keys()}
print(readable_failures)

Generating Animations/Visualizations

In [ ]:
terrain_idx = 13
apex_idx = 0

code, body_poses, foot_poses, plans, num_steps_hit, total_odes = evaluation.testPlannerSingle(robot,
                                                                                              rnn_astar_planner,
                                                                                              step_controller, 
                                                                                              step_test_matrix2,
                                                                                              3,
                                                                                              terrain_idx,
                                                                                              apex_idx,
                                                                                              friction = 0.8)

funcs = step_test_matrix2.getFunctions()
func = funcs[terrain_idx]
name = anim_dir + "rnn_astar_planner_step2_" + str(terrain_idx) + "_" + str(apex_idx) + ".mp4"

utils.animateMovingXAxis(body_poses, foot_poses, name, plans, nogo=None, terrain_func = func, fps = 50)

###RNN planner

In [ ]:
rnn_planner = rnn_planners.ConvRNNPlanner(model, device, -3, T = 1)
robot = hopper.Hopper(hopper.Constants())
perturbed_controller = deadbeat_net.PerturbedStepController(step_controller, 0.00)
rnn_test_results = evaluation.testRNNPlannerOnMatrix(robot,
                                                     rnn_planner,
                                                     perturbed_controller,
                                                     step_test_matrix,
                                                     time_to_replan = 3,
                                                     friction = 0.8,
                                                     tstep = 0.01)
print()
rnn_test_results.printMetrics()
readable_failures = {hopper.sim_codes_rev[i]:rnn_test_results.failure_cases[i] for i in rnn_test_results.failure_cases.keys()}
print(readable_failures)

# Miscellaneous

## Qualitatively evaluating the model

In [ ]:
idx = 8
test_matrix = ditch_test_matrix
terrain_array = test_matrix.arrays[idx]
terrain_func = test_matrix.getFunctions()[idx]
prev_steps = [[0.0]]
tnf = lambda x: np.pi/2
initial_apex = [0, 1.0, 0]
outs, softmaxes, hiddens = models.evaluateConvModel(model, 2, initial_apex, prev_steps, terrain_array, device, T = 1)
print(outs)

[0.20000004768371582, 0.7000000476837158]


In [ ]:
softmax_np = softmaxes[-1].cpu().detach().numpy()[0][0]
utils.plotProbabilitiesOverTerrain(outs,
                                  initial_apex,
                                  softmax_np,
                                  terrain_func,
                                  None)

## Generating data for the deadbeat controller

In [ ]:
def generateStepData(num_vels, num_angles, min_vi, max_vi, min_y, max_y, friction = 1.5):
  input_velocities = np.linspace(min_vi, max_vi, num_vels)
  input_ys = np.linspace(min_y, max_y, 50)
  vels, ys = np.meshgrid(input_velocities, input_ys)
  vels = np.ravel(vels)
  ys = np.ravel(ys)
  velocities = []
  angles = []
  min_limit = np.pi/2 - np.arctan(friction)
  max_limit = np.pi/2 + np.arctan(friction)
  terrain_func = lambda x: 0
  terrain_normal_func = lambda x: np.pi/2
  for i in range(len(vels)):
    if i%100 == 0:
      print(i, "/", len(vels))
    vel = vels[i]
    y = ys[i]
    input_angles = np.linspace(min_limit, max_limit, num_angles)
    for inp in input_angles:
      x0_apex = [0, y, vel, 0, 0, inp]
      apex1, apex2, last_flight = hopper.getNextState2(x0_apex, inp, lambda x: 0, lambda x: np.pi/2, friction)
      if apex1 is not None:
        velocities.append([vel, y, last_flight[0]])
        angles.append(inp)
  return np.array(velocities), np.array(angles)

In [ ]:
# dir = "/content/drive/My Drive/Research/legged_planning_learning/"
data_fname = dataset_dir + "deadbeat_datapoints.npy"
lab_fname = dataset_dir + "deadbeat_labels.npy"

inputs, labels = generateStepData(100, 100, -3, 3, 0.6, 1.5, friction = 0.8)
np.save(glob_dir + data_fname, inputs)
np.save(glob_dir + lab_fname, labels)

## Training the controller net


In [ ]:
# dir = "/content/drive/My Drive/Research/legged_planning_learning/"
data_fname = "deadbeat_datapoints.npy"
lab_fname = "deadbeat_labels.npy"

deadbeat_dset = np.load(glob_dir + data_fname)
deadbeat_labels = np.load(glob_dir + lab_fname)

In [ ]:
train_pct = 0.9
train_length = int(deadbeat_dset.shape[0] * train_pct)
test_length = deadbeat_dset.shape[0] - train_length

full_torch_dset = torch.utils.data.TensorDataset(torch.from_numpy(deadbeat_dset).float(),
                                     torch.from_numpy(deadbeat_labels).float())
trainset, testset = torch.utils.data.random_split(full_torch_dset, [train_length, test_length])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                              shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                              shuffle=True)

In [ ]:
db_net = deadbeat_net.DeadbeatNet()
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("using GPU")
else:
    device = torch.device("cpu")
    print("warning: using cpu")
db_net = db_net.to(device)

lr = 1e-3
n_epochs = 100
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(db_net.parameters(), lr=lr)

In [ ]:
# training loop for RNN
for epoch in range(1, n_epochs):
  losses = []
  # iterating over batches
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    output = torch.squeeze(db_net(inputs))
    loss = criterion(output, labels)
    loss.backward() 
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss.item())
    if i % 100 == 0:
      print("Epoch", epoch, "Batch ", i, "Loss:", loss.item())
  
  print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
  print("Loss: {:.4f}".format(np.mean(losses)))

In [ ]:
# testing loop
db_net = db_net.eval()
test_loss = 0
num_batches = 0
for i, data in enumerate(testloader, 0):
  inputs, labels = data
  inputs = inputs.to(device)
  labels = labels.to(device)
  output = torch.squeeze(db_net(inputs))
  loss = criterion(output, labels)
  num_batches += 1
  test_loss += loss.item()

avg_test_loss = test_loss/num_batches
print("test loss = ", avg_test_loss)

In [ ]:
torch.save(db_net.state_dict(), glob_dir + "deadbeat_net_state_1104")